## Interactive VLF data miner
### Vojtěch Laitl 2016
#### Ionozor Measuring Network - VLF monitors data analysis and graph processing
This is a iPython Jupyter notebook script made for data tooling and graph processing based on user's request.

In [ ]:
%Initial environment
disp(['This is a data browsing script made for SID monitors.'])
disp(['Choose your requests, please.'])
disp(['1. Choose an observatory (observatory = "your_request"; e.g. observatory = "VLFOSTROV).'])
disp(['2. Choose a station (station = "your_request"; e.g. station = "R1").'])
disp(['3. Set a date where you would like to show data from (date = "YY/MM/DD"; e.g. date = "2016/06/27").'])
disp(['Set your request for the initial time (H0 = hours, M0 = minutes, S0 = seconds; e.g. H0 = 12.'])
disp(['Set your request for the last-gap time (H1 = hours, M1 = minutes, S1 = seconds; e.g. M1 = 39.)'])

The first process opens the notebook and shows the user the basic instructions how to implement the appropriate diagnostic.
Users have the choice to set the observatory, the station, date and time so far.

In [ ]:
preambule = "space.astro.cz/ionozor/VLF/"
slash = "/"
spec = "spec"
format = ".h5"
sid = "sid_"
under = "_"
path = [preambule observatory slash station slash date slash sid observatory under station under spec format]
data = urlread(path)
disp(['Your requests have been sent succesfully. Now, you can choose some of the providen diagnostics to show the data.'])
disp(['The diagnostics you can choose from are:'])
disp(['1. VLF spectrum;'])
disp(['2. Ionogram;'])
disp(['3. 3D ionospheric model or'])
disp(['4. High-energy impact spectra.'])
disp(['Some brief introduction to each of them is given in the iPython Jupyter notebook.'])
disp(['If you are happy with the diagnostics, you can choose the appropriate one (diagnostic = "your_request; e.g. diagnostic = "VLF spectrum").'])

In [ ]:
t0 = data(1,:)
stringt = (1:columns(t))
t = (t0 + stept.*(stringt-1))'
t0 = ceil(H0./24.*rows(t))
t1 = floor(H1./24.*row(t))
t = t(,t0:t1)
nu0 = data(:,1)
stringnu = ((1:rows(nu)))'
nu = nu0 + stepnu.*(stringnu-1)
L0 = data(2:,t0:t1)
L = mean(L0,'g')
epsilon = 8.8542*10^-12
e = 1.602*10^-19
k_B = 1.38*10^-23
gamma = 10^-3
R_inf = 3.2899*10^15
c = 3*10^8
b = 2.898*10^-3
h = 6.626*10^-34
m_el = 9.109*10^-31
R = 8.314
N_A = 6.022*10^23

This code specifies the basic dependence invariant to all of the diagnostics.

In [ ]:
if diagnostic = "VLF spectrum"
plot(L,";15~80 kHz in time t0~t;")
axis('labely')
ylabel"Radio signal intensity level [dB]"
VLF_spectrum = [t nu L]
save -hdf5 VLF_spectrum.h5 VLFspectrum
else
disp(['VLF spectrum is empty.'])
endif
if diagnostic = "Ionogram"
dt = [0 [diff(t)]]
L1 = 0.1.*L
n0 = -1/4.*L1.^-4
E_k = - (36*pi)^-2.*epsilon^-4/3.*n/e^-4
T = -E_k./k_B
lambda_De = sqrt((epsilon.*k_B.*T)./(n.*e^2))
save -hdf5 debye_radius.h5 lambda_De
p = [0 [diff(E_k)]]
nu_delta = p.*c./h
omega = nu_delta + nu
n_0 = (nu.^2*m_el*epsilon)/e^2
n = -1/4.*L1.^-4 + n_0
dn = [0 [diff(n)]]
n0 = 23400^2*epsilon*m_el/e^2
N = n-n0
Bl = 1+(0.01.*L)
mD = 10^-10
h0 = abs(pi*i.*Bl./(2+2.*mD))
h0_t = (10.*h0).^2./10
f_n = dn.*Bl./(N.*(1-mD))
f_n2 = f_n.^2
f_ln = log(1/2.*f_n-((f_n2.^2 - 4).^1/2))
h_t = abs((Bl.*f_ln + 2.*pi.*i)./(1-mD))
H = 10.*(h0_t + h_t)
save -hdf5 height.h5 H
Bl1 = 10+(0.1.*L)
f_H = H.*(1-mD)./Bl1
exp1 = exp(f_H./10^11)
exp2 = exp(-f_H./10^11)
f_B = (1-mD)./Bl1
n_H = L.^-5 + 4.*N.*f_B.*(exp1 + exp2)
ionogram = [t H n n_H]
save -hdf5 ionogram.h5 ionogram
plot3(t,n_H,H)
axis('labely','labelz')
ylabel"dn/dL + dn/dH [1/m3]"
zlabel"Height [km]"
else
disp(['Ionogram is empty.'])
endif
if diagnostic = 3D ionospheric model
E_k = - (36*pi)^-2.*epsilon^-4/3.*n/e^-4
T = -E_k./k_B
lambda_De = sqrt((epsilon.*k_B.*T)./(n.*e^2))
save -hdf5 debye_radius.h5 lambda_De
p = [0 [diff(E_k)]]
nu_delta = p.*c./h
omega = nu_delta + nu
n_0 = (nu.^2*m_el*epsilon)/e^2
n = -1/4.*L1.^-4 + n_0
dn = [0 [diff(n)]]
n0 = 23400^2*epsilon*m_el/e^2
N = n-n0
Bl = 1+(0.01.*L)
mD = 10^-10
h0 = abs(pi*i.*Bl./(2+2.*mD))
h0_t = (10.*h0).^2./10
f_n = dn.*Bl./(N.*(1-mD))
f_n2 = f_n.^2
f_ln = log(1/2.*f_n-((f_n2.^2 - 4).^1/2))
h_t = abs((Bl.*f_ln + 2.*pi.*i)./(1-mD))
H = 10.*(h0_t + h_t)
save -hdf5 height.h5 H
Bl1 = 10+(0.1.*L)
f_H = H.*(1-mD)./Bl1
exp1 = exp(f_H./10^11)
exp2 = exp(-f_H./10^11)
f_B = (1-mD)./Bl1
n_H = L.^-5 + 4.*N.*f_B.*(exp1 + exp2)
save -hdf5 height_dependent_conc.h5 n_H
plot3(t,n_H,H)
axis('labely','labelz')
ylabel"dn/dL + dn/dH [1/m3]"
zlabel"Height [km]"
I0 = -26.74
R = 1.496*10^11
Is = exp(L./10)
fce = Is.*R.^2./I0
alpha = acos(fce)
x = abs(H.*tan(alpha))
drift = [t H x alpha]
save -hdf5 drift.h5 drift
T = [t t]
X = [x -x]
H1 = [H H]
mesh(T,X,H1)
axis('labely','labelz')
ylabel"Horizontal flow [km]"
zlabel"Height [km]"
else
disp(['3D ionospheric model is empty.'])
endif
if diagnostic = "High-energy impact spectra"
E_k = - (36*pi)^-2.*epsilon^-4/3.*n/e^-4
T = -E_k./k_B
lambda_De = sqrt((epsilon.*k_B.*T)./(n.*e^2))
save -hdf5 debye_radius.h5 lambda_De
p = [0 [diff(E_k)]]
nu_delta = p.*c./h
omega = nu_delta + nu
n_0 = (nu.^2*m_el*epsilon)/e^2
n = -1/4.*L1.^-4 + n_0
dn = [0 [diff(n)]]
n0 = 23400^2*epsilon*m_el/e^2
N = n-n0
Bl = 1+(0.01.*L)
mD = 10^-10
h0 = abs(pi*i.*Bl./(2+2.*mD))
h0_t = (10.*h0).^2./10
f_n = dn.*Bl./(N.*(1-mD))
f_n2 = f_n.^2
f_ln = log(1/2.*f_n-((f_n2.^2 - 4).^1/2))
h_t = abs((Bl.*f_ln + 2.*pi.*i)./(1-mD))
H = 10.*(h0_t + h_t)
Bl1 = 10+(0.1.*L)
f_H = H.*(1-mD)./Bl1
exp1 = exp(f_H./10^11)
exp2 = exp(-f_H./10^11)
f_B = (1-mD)./Bl1
n_H = L.^-5 + 4.*N.*f_B.*(exp1 + exp2)
p = h0.*(mD-1)
p1 = H.*(1-mD) + pi
n_el = 10.*(exp((2.*N_D./1000.*(p1-p) + Bl)./Bl) + 4.*H)
n_el1 = n_el./10^6
T_e = 3^2/3.*e^2.*n_el.^1/3.*N_D.^2/3./(epsilon*k_B*(4*pi)^2/3).*lambda_De.^1.2
n_e = 10.*(exp((2.*N_D./1000000.*(p1-p) + Bl)./Bl) + 4.*H)
n_e1 = n_e./10^6
T_EL = 3^2/3.*e^2.*n_e.^1/3.*N_D.^2/3./(epsilon*k_B*(4*pi)^2/3).*lambda_De.^1.2
lambda_M = (h*c.*lambda_De.^(N_D./100))./(k_B.*T_e)
lambda_S = (h*c.*lambda_De.^(N_D./100))./(k_B.*T_EL)
f_nh = (-2.*H.*(mD-1) + h0_t.*(-mD) + h0_t + pi)./B
f_exp = exp(f_nh)
f_T = (-2^4/3.*(mD-1).*N.*(1-mD).*log10(2.71818281).*f_exp)./(3.*B.^2.*N.*(1-mD).*f_exp)
dT = 3^2/3.*e^2.*N_D.^2/3./(epsilon.*k_B.*(4*pi)^2/3).*lambda_De.^6/5.*f_T
dTe = [0 [diff(T_e)]]
Z = -1./dTe.*dT
f1 = n_el.*E_a.*e./Z
f2 = k_B.*T_e
f_I = f1.*f2.*exp(-T_el./T_e)
I = abs(log(f_I))
T = 10.*abs(log(b./lambda_M))
dTe1 = [0 [diff(T)]
lambda = b./T
Z = log(((1.5*R_inf.*sqrt(3*R_inf*c./lambda))./(1.5*R_inf)))-9
dT = abs(10.*(log(T)))
n = (16.*epsilon^3.*k_B^3.*pi^2.*lambda_De.^1.5.*exp(dT./(0.1.*N_D)))./(9.*e^6.*N_D.^2)
Z = -1./dTe1.*dT
f1 = n.*E_a.*e./Z
f2 = k_B.*T
f_I = f1.*f2.*exp(-T_el./T)
IM = abs(log(f_I))
spectra = [t lambda IM]
save -hdf5 HE_impact_spectra.h5 spectra
plot3(t,lambda,IM)
axis('labely','labelz')
ylabel"Wavelength [m]"
zlabel"Intensity [a.u.]"
else
disp(['High-energy impact spectra are empty.'])
endif
disp(['Congratulations! You have managed to have drawn your diagnostic!'])
disp(['The data were saved to the local disc. Would you also like to save the graph (ans = "y/n"; e.g. ans = "y").'])
if ans = "y"
print 'My_diagnostic.png','-dpng','-landscape','-FArial:18'
else
disp(['Your graph has not been saved. You can now leave the mode by shutting down Jupyter Notebook.'])
endif

The code written higher computes the diagnostics specified or chosen by the users. They have also the choice to set if they want to print the graph or not.

The alternative solution is given at `https://github.com/Ionozor/data-processing/tree/master/Octave/bin/User%20data%20mining`.